In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import math
import time
import string


In [4]:
# Sklearn libraries for machine learning and text processing
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

# NLTK libraries for text processing (lemmatization, stemming, stopwords, POS tagging)
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer, PorterStemmer

%matplotlib inline
sns.set_style("whitegrid")
plt.style.use("fivethirtyeight")

# Download necessary NLTK resources for text processing
nltk.download('wordnet')  # WordNet for lemmatization
nltk.download('omw-1.4')  # Open Multilingual Wordnet
nltk.download('punkt')  # Tokenizer
nltk.download('stopwords')  # Stopwords for text cleaning
nltk.download('averaged_perceptron_tagger')  # POS tagger for part-of-speech tagging
nltk.download('tagsets_json')  # Tagset resource


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package tagsets_json to /root/nltk_data...
[nltk_data]   Package tagsets_json is already up-to-date!


True

In [5]:
from google.colab import files
uplaoded = files.upload()

Saving en_train.csv to en_train (1).csv


In [6]:
df_train = pd.read_csv('en_train.csv')
print(df_train.head(5))

                                                text    binary  \
0  #USER# #USER# #USER# #USER# You expect a man t...  Not Hope   
1  #USER# #USER# #USER# #USER# Tinubu is actually...  Not Hope   
2  it'd be nice if missguided actually had stock ...      Hope   
3  #USER# Anyway love u bubbly i know i can count...      Hope   
4  “you have a lot of people rooting for you whet...  Not Hope   

         multiclass  
0          Not Hope  
1          Not Hope  
2           Sarcasm  
3  Generalized Hope  
4          Not Hope  


In [7]:
from google.colab import files
uplaoded = files.upload()

Saving en_dev.csv to en_dev (1).csv


In [8]:
df_test = pd.read_csv('en_dev.csv')
print(df_test.head(5))

                                                text    binary  \
0  And you’d come to the obvious logical solution...      Hope   
1  #USER# I’m hoping his type helps quiet our wor...      Hope   
2  I aspire to be his sugar mommy...I know chance...      Hope   
3  also that jacket he wore yesterday i���m gonna...      Hope   
4  How can they visually express themselves witho...  Not Hope   

         multiclass  
0  Generalized Hope  
1    Realistic Hope  
2  Unrealistic Hope  
3           Sarcasm  
4           Sarcasm  


In [10]:
df_train["text"]

,text
0,#USER# #USER# #USER# #USER# You expect a man t...
1,#USER# #USER# #USER# #USER# Tinubu is actually...
2,it'd be nice if missguided actually had stock ...
3,#USER# Anyway love u bubbly i know i can count...
4,“you have a lot of people rooting for you whet...
...,...
5228,naming my son onickolas in honor of my love fo...
5229,The Hanging Chad of the CIA Crowley Bush Famil...
5230,the entire history of human desire takes about...
5231,#USER# #USER# Not true. I just donated to one ...


In [11]:
df_train.describe()

,text,binary,multiclass
count,5233,5233,5233
unique,5233,2,5
top,#USER# i hope that that one person didn't ruin...,Not Hope,Not Hope
freq,1,2807,2245


In [12]:

df_train.groupby('binary').describe()

text                                                                 \
         count unique                                                top freq   
binary                                                                          
Hope      2426   2426  #USER# i hope that that one person didn't ruin...    1   
Not Hope  2807   2807  #USER# #USER# Not true. I just donated to one ...    1   

         multiclass                                 
              count unique               top  freq  
binary                                              
Hope           2426      4  Generalized Hope  1284  
Not Hope       2807      2          Not Hope  2245

In [13]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
import string
from nltk.corpus import stopwords

def text_process(tweet):
    """
    Cleans and processes a tweet string.

    This function takes a raw tweet string and performs several text cleaning steps:
    1. Checks if the input is a string; returns an empty string otherwise.
    2. Defines a list of common English stopwords and additional custom words to remove.
    3. Removes all punctuation from the tweet.
    4. Removes the defined stopwords from the tweet.
    5. Joins the remaining words back into a single cleaned string.

    Args:
        tweet (str): The input tweet string to be processed.

    Returns:
        str: The cleaned tweet string with punctuation and stopwords removed,
             or an empty string if the input was not a string.
    """
    # Handle non-string values gracefully
    if not isinstance(tweet, str):
        return ""
    tweet = tweet.lower()

    # Define a set of stopwords, including standard English stopwords and some custom ones
    STOPWORDS = stopwords.words('english') + ['u', 'ü', 'ur', '4', '2', 'im', 'dont', 'doin', 'ure']

    # Remove punctuation from the tweet
    nopunc = ''.join([char for char in tweet if char not in string.punctuation])

    # Remove stopwords and join the words back into a string
    return ' '.join([word for word in nopunc.split() if word.lower() not in STOPWORDS])

In [15]:
df_train['clean_tweets'] = df_train['text'].apply(text_process)

In [16]:
df_train.head(15)

,text,binary,multiclass,clean_tweets
0,#USER# #USER# #USER# #USER# You expect a man t...,Not Hope,Not Hope,user user user user expect man literally refer...
1,#USER# #USER# #USER# #USER# Tinubu is actually...,Not Hope,Not Hope,user user user user tinubu actually bonus lols...
2,it'd be nice if missguided actually had stock ...,Hope,Sarcasm,itd nice missguided actually stock once�������...
3,#USER# Anyway love u bubbly i know i can count...,Hope,Generalized Hope,user anyway love bubbly know count fairy tail😌🤚
4,“you have a lot of people rooting for you whet...,Not Hope,Not Hope,“you lot people rooting whether believe should...
5,Hoping to get on the brown a lot more after I ...,Not Hope,Not Hope,hoping get brown lot move part city lot bars
6,#USER# no exactly :/ and like i’ve seen (non g...,Not Hope,Not Hope,user exactly like i’ve seen non gvfx people sa...
7,#USER# You have lost because your so called cu...,Not Hope,Not Hope,user lost called culture backward never progre...
8,#USER# #USER# #USER# #USER# #USER# #USER# #USE...,Hope,Realistic Hope,user user user user user user user user hi😊 i’...
9,#USER# #USER# My dear Rwandans still remember ...,Not Hope,Not Hope,user user dear rwandans still remember happene...


In [17]:
# Apply text cleaning to the "text" column
df_train['clean_tweets'] = df_train['text'].apply(text_process)
df_test['clean_tweets'] = df_test['text'].apply(text_process)

# 2. Keep only binary labels (negative / positive)
# Filter the training data to keep only 'negative' and 'positive'
binary_mapping = {'Not Hope': 0, 'Hope': 1}

# Create a copy to avoid SettingWithCopyWarning
df_train = df_train[df_train['binary'].isin(binary_mapping.keys())].copy()

# Map sentiment labels to numeric values
df_train['binary_num'] = df_train['binary'].map(binary_mapping)
print(df_train)

                                                   text    binary  \
0     #USER# #USER# #USER# #USER# You expect a man t...  Not Hope   
1     #USER# #USER# #USER# #USER# Tinubu is actually...  Not Hope   
2     it'd be nice if missguided actually had stock ...      Hope   
3     #USER# Anyway love u bubbly i know i can count...      Hope   
4     “you have a lot of people rooting for you whet...  Not Hope   
...                                                 ...       ...   
5228  naming my son onickolas in honor of my love fo...      Hope   
5229  The Hanging Chad of the CIA Crowley Bush Famil...  Not Hope   
5230  the entire history of human desire takes about...  Not Hope   
5231  #USER# #USER# Not true. I just donated to one ...  Not Hope   
5232  #USER# i hope that that one person didn't ruin...      Hope   

            multiclass                                       clean_tweets  \
0             Not Hope  user user user user expect man literally refer...   
1             Not

In [18]:
# 3. Show UNIQUE-WORD statistics
# Fix: Use 'clean_tweets' instead of 'clean'
train_vocab = set(" ".join(df_train['clean_tweets']).split()) # set -> unique word
test_vocab = set(" ".join(df_test['clean_tweets']).split())

print("=== Vocabulary statistics ===")
print(f"Train unique words : {len(train_vocab):,}")
print(f"Test  unique words : {len(test_vocab):,}")
print(f"Overlap             : {len(train_vocab & test_vocab):,} (common words that appear in both the training dataset and the test dataset)\n")
print(f"Test OOV            : {len(test_vocab - train_vocab):,} (words in test but not in train)\n")

=== Vocabulary statistics ===
Train unique words : 18,868
Test  unique words : 10,010
Overlap             : 5,867 (common words that appear in both the training dataset and the test dataset)

Test OOV            : 4,143 (words in test but not in train)



In [19]:
print("Sample train words :", sorted(list(train_vocab))[:10], "...\n")
print("Sample test  words :", sorted(list(test_vocab))[:10], "...\n")

Sample train words : ['0', '00', '0000', '000268958', '005', '007', '00700', '01', '010', '015'] ...

Sample test  words : ['0', '005', '02', '08080790505', '1', '10', '100', '1000', '1000000', '100400'] ...



In [20]:
x = df_train['clean_tweets']
y = df_train['binary']
print("shape x : " , x.shape)
print("shape y : " , y.shape)

shape x :  (5233,)
shape y :  (5233,)


In [21]:
# 4. Train / validation split
# Split training data into train and validation sets
from sklearn.model_selection import train_test_split
x_train, x_test, y_train_lable, y_test_lable = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

In [22]:
# Machine Learning Model Training and Testing

#CountVectorizer: This method represents text based on the frequency of words.
#TF-IDF (Term Frequency-Inverse Document Frequency): This method weighs words based on their importance in a document relative to the entire corpus.

In [23]:
# Countvectorizer

# Instantiate CountVectorizer with binary=True and fit on training data
vectorizer = CountVectorizer(binary=True)
x_train_data = vectorizer.fit_transform(x_train) # -> train modle on training data
x_test_data = vectorizer.transform(x_test) # -> testing modle on test data

print("/n vextorizer size of vocab : ", len(vectorizer.get_feature_names_out()))
print("Sample features:", vectorizer.get_feature_names_out()[:20])

/n vextorizer size of vocab :  15249
Sample features: ['00' '000268958' '005' '00700' '01' '010' '015' '02' '020' '025' '03'
 '04' '05' '06' '0625' '06272022' '0n'
 '0xfc6806d43cf176a6419be51bfdc575406a018cd6' '10' '100']


In [24]:
from sklearn.naive_bayes import MultinomialNB

# Train the Multinomial Naive Bayes model
start_time = time.time()
nb = MultinomialNB()
nb.fit(x_train_data, y_train_lable) # -> check how many time whitch will take to train modle 80% data and his lable
end_time = time.time()

print("\nTraining Time: {:.3f} seconds".format(end_time - start_time))


Training Time: 0.014 seconds


In [25]:
# 7. Evaluate
# Make predictions on the validation set
x_test_pred_class = nb.predict(x_test_data)
x_test_pred_class

array(['Hope', 'Hope', 'Not Hope', ..., 'Not Hope', 'Not Hope', 'Hope'],
      dtype='<U8')

In [26]:
y_test_lable

,binary
1145,Not Hope
4228,Not Hope
2894,Not Hope
3570,Not Hope
4091,Hope
...,...
2358,Not Hope
3684,Not Hope
2959,Not Hope
2196,Not Hope


In [27]:
from sklearn import metrics

print("\n=======Accuracy Score===========")
acc = metrics.accuracy_score(y_test_lable , x_test_pred_class)
print(f"\nMultinomialNB Accuracy : {acc*100:.2f}%")

# Print the confusion matrix
print("\n=======Confusion Matrix===========")
print(metrics.confusion_matrix(y_test_lable , x_test_pred_class))

# Additional evaluation metrics
print("\n=======Classification Report===========")
#print(metrics.classification_report(y_test_lable , x_test_pred_class, target_names=['Negative', 'Positive']))
print(metrics.classification_report(y_test_lable, x_test_pred_class, target_names=['Not Hope', 'Hope']))


=======Accuracy Score===========

MultinomialNB Accuracy : 76.41%

=======Confusion Matrix===========
[[369 116]
 [131 431]]

=======Classification Report===========
              precision    recall  f1-score   support

    Not Hope       0.74      0.76      0.75       485
        Hope       0.79      0.77      0.78       562

    accuracy                           0.76      1047
   macro avg       0.76      0.76      0.76      1047
weighted avg       0.76      0.76      0.76      1047



In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(
    max_features = 20000,
    ngram_range = (1,2),
    min_df = 2,
    max_df = 0.95
)

In [29]:
x_train_tfidf = vectorizer.fit_transform(x_train)
x_test_tfidf= vectorizer.transform(x_train)
print("\nTF-IDF Vectorizer vocabulary size:", len(vectorizer.get_feature_names_out()))
print("Sample features:", vectorizer.get_feature_names_out()[:5])


TF-IDF Vectorizer vocabulary size: 9611
Sample features: ['00' '01' '010' '02' '03']


In [30]:
print(x_train.shape)       # original text
print(x_train_tfidf.shape) # after TF-IDF vectorization


(4186,)
(4186, 9611)


In [31]:
print(df_train.shape)
print(y_train_lable.shape)


(5233, 5)
(4186,)


In [32]:
# First do the split
x_train, x_test, y_train, y_test = train_test_split(
    df_train["clean_tweets"], df_train["binary_num"],
    test_size=0.2, random_state=42, stratify=df_train["binary_num"]
)

# Then vectorize only training data
vectorizer = TfidfVectorizer()
x_train_tfidf = vectorizer.fit_transform(x_train)
x_test_tfidf = vectorizer.transform(x_test)

# Then fit model
nb = MultinomialNB()
nb.fit(x_train_tfidf, y_train.values.ravel())


MultinomialNB()

In [33]:

# Make predictions on the validation set
y_pred_class_tfidf = nb.predict(x_test_tfidf)

In [34]:
y_pred_class_tfidf

array([1, 1, 1, ..., 1, 1, 0])

In [35]:
label_mapping = {'Hope': 1, 'Not Hope': 0}  # or the opposite if your training was opposite
y_test_lable_numeric = y_test_lable.map(label_mapping)

In [36]:

from sklearn import metrics

# Accuracy Score
acc1 = metrics.accuracy_score(y_test_lable_numeric, y_pred_class_tfidf)
print("\n=======Accuracy Score===========")
print(f"\nMultinomialNB Accuracy : {acc1*100:.2f}%")

# Confusion Matrix
print("\n=======Confusion Matrix===========")
print(metrics.confusion_matrix(y_test_lable_numeric, y_pred_class_tfidf))

# Classification Report
print("\n=======Classification Report===========")
print(metrics.classification_report(y_test_lable_numeric, y_pred_class_tfidf, target_names=['Not Hope', 'Hope']))



=======Accuracy Score===========

MultinomialNB Accuracy : 29.32%

=======Confusion Matrix===========
[[238 324]
 [416  69]]

=======Classification Report===========
              precision    recall  f1-score   support

    Not Hope       0.36      0.42      0.39       562
        Hope       0.18      0.14      0.16       485

    accuracy                           0.29      1047
   macro avg       0.27      0.28      0.27      1047
weighted avg       0.28      0.29      0.28      1047



In [37]:
# Optional: Show most informative features using log probabilities
print("\n=======Most Important Features===========")
feature_names = vectorizer.get_feature_names_out()

# Get log probabilities for each class
log_prob_positive = nb.feature_log_prob_[1]  # Class 1 (positive)
log_prob_negative = nb.feature_log_prob_[0]  # Class 0 (negative)

# Calculate the difference (higher = more indicative of positive sentiment)
feature_scores = log_prob_positive - log_prob_negative

# Get top features for each class
top_positive_idx = feature_scores.argsort()[-10:][::-1]  # Top 10 positive features
top_negative_idx = feature_scores.argsort()[:10]         # Top 10 negative features

print("Top features for POSITIVE sentiment:")
for i in top_positive_idx:
    print(f"  {feature_names[i]}: {feature_scores[i]:.4f}")

print("\nTop features for NEGATIVE sentiment:")
for i in top_negative_idx:
    print(f"  {feature_names[i]}: {feature_scores[i]:.4f}")



=======Most Important Features===========
Top features for POSITIVE sentiment:
  hope: 2.5505
  hoping: 2.0974
  hopeful: 1.7757
  station: 1.4845
  luck: 1.4827
  hi: 1.4386
  yettocome: 1.4224
  happiness: 1.3634
  soon: 1.3229
  congratulations: 1.2980

Top features for NEGATIVE sentiment:
  believe: -1.5012
  expect: -1.4492
  view: -1.4299
  child: -1.4117
  simply: -1.3983
  supposed: -1.3967
  saying: -1.3683
  speaking: -1.3637
  force: -1.3542
  gay: -1.3526


In [38]:
# Logistic Regression using TF-IDF

In [39]:
# Instantiate TfidfVectorizer and fit on training data
vectorizer = TfidfVectorizer(
    max_features=10000,  # Limit vocabulary size for efficiency
    ngram_range=(1, 2),  # Use unigrams and bigrams
    min_df=2,           # Ignore terms that appear in less than 2 documents
    max_df=0.95         # Ignore terms that appear in more than 95% of documents
)
X_train_tfidf = vectorizer.fit_transform(x_train)
X_val_tfidf = vectorizer.transform(x_test)

print("\nTF-IDF Vectorizer vocabulary size:", len(vectorizer.get_feature_names_out()))
print("Sample features:", vectorizer.get_feature_names_out()[:20])



TF-IDF Vectorizer vocabulary size: 9653
Sample features: ['00' '007' '01' '010' '03' '05' '0625' '06272022' '06272022 total' '10'
 '10 count' '10 years' '100' '100 effective' '100 week' '100 years' '1000'
 '10000' '105' '11']


In [40]:
# Train both Multinomial Naive Bayes and Logistic Regression models
start_time = time.time()

# Naive Bayes
nb = MultinomialNB()
nb.fit(X_train_tfidf, y_train)

# Logistic Regression
lr = LogisticRegression(random_state=42, max_iter=1000)
lr.fit(X_train_tfidf, y_train)

end_time = time.time()
print("\nTraining Time: {:.3f} seconds".format(end_time - start_time))



Training Time: 0.056 seconds


In [41]:
label_mapping = {'Not Hope': 0, 'Hope': 1}
y_test_lable_mapped = y_test_lable.map(label_mapping)


In [42]:
# 7. Evaluate both models
# Make predictions on the validation set
y_pred_nb = nb.predict(X_val_tfidf)
y_pred_lr = lr.predict(X_val_tfidf)

# Calculate and print accuracy scores
print("\n=======Accuracy Scores===========")
acc_nb = metrics.accuracy_score(y_test_lable_mapped, y_pred_nb)
acc_lr = metrics.accuracy_score(y_test_lable_mapped, y_pred_lr)

print(f"Naive Bayes Accuracy         : {acc_nb*100:.2f}%")
print(f"Logistic Regression Accuracy : {acc_lr*100:.2f}%")

# Print confusion matrices
print("\n=======Confusion Matrix - Naive Bayes===========")
print(metrics.confusion_matrix(y_test_lable_mapped, y_pred_nb))

print("\n=======Confusion Matrix - Logistic Regression===========")
print(metrics.confusion_matrix(y_test_lable_mapped, y_pred_lr))





=======Accuracy Scores===========
Naive Bayes Accuracy         : 27.89%
Logistic Regression Accuracy : 26.27%

=======Confusion Matrix - Naive Bayes===========
[[218 344]
 [411  74]]

=======Confusion Matrix - Logistic Regression===========
[[190 372]
 [400  85]]


In [43]:


label_mapping = {'Not Hope': 0, 'Hope': 1}
y_test_lable_mapped = y_test_lable.map(label_mapping)

# Classification reports
print("\n=======Classification Report - Naive Bayes===========")
print(metrics.classification_report(y_test_lable_mapped, y_pred_nb, target_names=['Negative', 'Positive']))

print("\n=======Classification Report - Logistic Regression===========")
print(metrics.classification_report(y_test_lable_mapped, y_pred_lr, target_names=['Negative', 'Positive']))

# Optional: Show most informative features using log probabilities
print("\n=======Most Important Features===========")
feature_names = vectorizer.get_feature_names_out()

# Get log probabilities for each class
log_prob_positive = nb.feature_log_prob_[1]  # Class 1 (positive)
log_prob_negative = nb.feature_log_prob_[0]  # Class 0 (negative)

# Calculate the difference (higher = more indicative of positive sentiment)
feature_scores = log_prob_positive - log_prob_negative

# Get top features for each class
top_positive_idx = feature_scores.argsort()[-10:][::-1]  # Top 10 positive features
top_negative_idx = feature_scores.argsort()[:10]         # Top 10 negative features

print("Top features for POSITIVE sentiment:")
for i in top_positive_idx:
    print(f"  {feature_names[i]}: {feature_scores[i]:.4f}")

print("\nTop features for NEGATIVE sentiment:")
for i in top_negative_idx:
    print(f"  {feature_names[i]}: {feature_scores[i]:.4f}")


=======Classification Report - Naive Bayes===========
              precision    recall  f1-score   support

    Negative       0.35      0.39      0.37       562
    Positive       0.18      0.15      0.16       485

    accuracy                           0.28      1047
   macro avg       0.26      0.27      0.26      1047
weighted avg       0.27      0.28      0.27      1047


=======Classification Report - Logistic Regression===========
              precision    recall  f1-score   support

    Negative       0.32      0.34      0.33       562
    Positive       0.19      0.18      0.18       485

    accuracy                           0.26      1047
   macro avg       0.25      0.26      0.26      1047
weighted avg       0.26      0.26      0.26      1047


=======Most Important Features===========
Top features for POSITIVE sentiment:
  hope: 2.5140
  hoping: 2.0793
  one day: 2.0078
  user hope: 1.9431
  heres hoping: 1.7472
  hopeful: 1.7334
  really hoping: 1.7274
  user hoping